In [ ]:
from fair import FAIR

In [ ]:
from fair.interface import fill, initialise
from fair.io import read_properties

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import copy

In [ ]:
import matplotlib.pyplot as pl

In [ ]:
f = FAIR(ch4_method='thornhill2021')

In [ ]:
f.define_time(1750, 2101, 1)

In [ ]:
scen_df = pd.read_csv('../data/c1_emissions_scenarios.csv')

In [ ]:
scen_labels = pd.unique(list(zip(scen_df.Model, scen_df.Scenario)))

In [ ]:
scenarios = [f'{scen[0]}___{scen[1]}' for scen in scen_labels]

In [ ]:
# do one at a time
scenarios = [scenarios[0]]
f.define_scenarios(scenarios)
fair_params_df = pd.read_csv('../data/ar6_calibration_ebm3.csv', index_col=0)

In [ ]:
f.define_configs(list(fair_params_df.index))

In [ ]:
# load up default list of species and their properties
species, properties = read_properties()

# drop aviation NOx and contrails as they are not in the future scenarios
species.remove('Contrails')    # not modelled in UKESM, I think
species.remove('NOx aviation') # which renders this redundant

del properties['Contrails']
del properties['NOx aviation']

# declare species and properties
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
hist_df = pd.read_csv('../data/historical_emissions.csv')

In [ ]:
hist_df

In [ ]:
emitted_species_pipeline_names = [var.split('|')[-1] for var in hist_df.Variable]

In [ ]:
emitted_species_pipeline_names

In [ ]:
species_name_mapping = {name: name for name in emitted_species_pipeline_names}

In [ ]:
# hand edits  target_dict[n_key] = target_dict.pop(key)
species_name_mapping['CO2 AFOLU'] = species_name_mapping.pop('MAGICC AFOLU')
species_name_mapping['CO2 FFI'] = species_name_mapping.pop('MAGICC Fossil and Industrial')
species_name_mapping['HFC-125'] = species_name_mapping.pop('HFC125')
species_name_mapping['HFC-134a'] = species_name_mapping.pop('HFC134a')
species_name_mapping['HFC-143a'] = species_name_mapping.pop('HFC143a')
species_name_mapping['HFC-152a'] = species_name_mapping.pop('HFC152a')
species_name_mapping['HFC-227ea'] = species_name_mapping.pop('HFC227ea')
species_name_mapping['HFC-23'] = species_name_mapping.pop('HFC23')
species_name_mapping['HFC-236fa'] = species_name_mapping.pop('HFC236fa')
species_name_mapping['HFC-245fa'] = species_name_mapping.pop('HFC245fa')
species_name_mapping['HFC-32'] = species_name_mapping.pop('HFC32')
species_name_mapping['HFC-365mfc'] = species_name_mapping.pop('HFC365mfc')
species_name_mapping['HFC-4310mee'] = species_name_mapping.pop('HFC4310mee')
species_name_mapping['CFC-11'] = species_name_mapping.pop('CFC11')
species_name_mapping['CFC-12'] = species_name_mapping.pop('CFC12')
species_name_mapping['CFC-113'] = species_name_mapping.pop('CFC113')
species_name_mapping['CFC-114'] = species_name_mapping.pop('CFC114')
species_name_mapping['CFC-115'] = species_name_mapping.pop('CFC115')
species_name_mapping['HCFC-22'] = species_name_mapping.pop('HCFC22')
species_name_mapping['HCFC-141b'] = species_name_mapping.pop('HCFC141b')
species_name_mapping['HCFC-142b'] = species_name_mapping.pop('HCFC142b')
species_name_mapping['Halon-1202'] = species_name_mapping.pop('Halon1202')
species_name_mapping['Halon-1211'] = species_name_mapping.pop('Halon1211')
species_name_mapping['Halon-1301'] = species_name_mapping.pop('Halon1301')
species_name_mapping['Halon-2402'] = species_name_mapping.pop('Halon2402')
species_name_mapping['c-C4F8'] = species_name_mapping.pop('cC4F8')

In [ ]:
# Jeebus, this is frustrating the lack of naming consistency
species_name_mapping_future = copy.deepcopy(species_name_mapping)
species_name_mapping_future['CO2 AFOLU'] = 'AFOLU'
species_name_mapping_future['CO2 FFI'] = 'Energy and Industrial Processes'
species_name_mapping_future['HFC-245fa'] = 'HFC245ca'  # this is an error?
species_name_mapping_future['HFC-4310mee'] = 'HFC43-10'

In [ ]:
# argh, and the frigging daft units
unit_convert = {specie: 1 for specie in species}
unit_convert['CO2 AFOLU'] = 1/1000
unit_convert['CO2 FFI'] = 1/1000
unit_convert['N2O'] = 1/1000

In [ ]:
for scenario in scenarios:
    for specie, pipeline_name in species_name_mapping.items():
        f.emissions.loc[
            dict(scenario=scenario, specie=specie, timepoints=np.arange(1750.5, 2015))
        ] = hist_df.loc[hist_df['Variable'].str.endswith(f"|{pipeline_name}"), '1750':'2014'].values.T * unit_convert[specie]
        model, scen = scenario.split('___')
        f.emissions.loc[
            dict(scenario=scenario, specie=specie, timepoints=np.arange(2015.5, 2101))
        ] = scen_df.loc[
            (scen_df['Variable'].str.endswith(f"|{species_name_mapping_future[specie]}")) &
            (scen_df['Scenario']==scen) &
            (scen_df['Model']==model),
            '2015':'2100'
        ].values.T * unit_convert[specie]

In [ ]:
f.emissions

In [ ]:
# get solar and volcanic forcing from AR6
ar6_forcing_df = pd.read_csv('../data/table_A3.3_historical_ERF_1750-2019_best_estimate.csv', index_col=0)
ar6_forcing_df

In [ ]:
# Ramp volcanic down to zero over a decade, following CMIP6 convention
volcanic_forcing = np.zeros(352)
volcanic_forcing[:270] = ar6_forcing_df['volcanic'].values
volcanic_forcing[269:281] = np.linspace(1, 0, 12) * volcanic_forcing[269]

In [ ]:
# Use a zero solar amplitude post-2019, to isolate anthropogenic warming signal
solar_forcing=np.zeros(352)
solar_forcing[:270] = ar6_forcing_df['solar'].values

In [ ]:
# Put volcanic forcing into FaIR
fill(
    f.forcing, 
    volcanic_forcing[:, None, None] * fair_params_df.loc[:, 'scale Volcanic'].values[None, None, :], 
    specie='Volcanic'
)

In [ ]:
# Put solar forcing into FaIR
trend_shape = np.ones(352)
trend_shape[:271] = np.linspace(0, 1, 271)

fill(f.forcing, 
     solar_forcing[:, None, None] * 
     fair_params_df.loc[:, 'solar_amplitude'].values.squeeze() + 
     trend_shape[:, None, None] * fair_params_df.loc[:, 'solar_trend'].values.squeeze(),
     specie='Solar'
)

In [ ]:
# Get default species configs
f.fill_species_configs()

# Climate response
fill(f.climate_configs['ocean_heat_capacity'], fair_params_df.loc[:,'c1':'c3'])
fill(f.climate_configs['ocean_heat_transfer'], fair_params_df.loc[:,'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[:,'epsilon'])
fill(f.climate_configs['gamma_autocorrelation'], fair_params_df.loc[:,'gamma'])
fill(f.climate_configs['stochastic_run'], False)

# carbon cycle
fill(f.species_configs['iirf_0'], fair_params_df.loc[:, 'r0'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_airborne'], fair_params_df.loc[:, 'rA'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_uptake'], fair_params_df.loc[:, 'rU'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_temperature'], fair_params_df.loc[:, 'rT'].values.squeeze(), specie='CO2')

# aerosol direct
for specie in ['BC', 'CH4', 'N2O', 'NH3', 'NOx', 'OC', 'Sulfur', 'VOC', 'Equivalent effective stratospheric chlorine']:
    fill(f.species_configs['erfari_radiative_efficiency'], fair_params_df.loc[:, f'ari {specie}'].values.squeeze(), specie=specie)

# aerosol indirect
fill(f.species_configs['aci_scale'], fair_params_df.loc[:, 'beta'].values.squeeze())
fill(f.species_configs['aci_shape'], fair_params_df.loc[:, 'shape_so2'].values.squeeze(), specie='Sulfur')
fill(f.species_configs['aci_shape'], fair_params_df.loc[:, 'shape_bc'].values.squeeze(), specie='BC')
fill(f.species_configs['aci_shape'], fair_params_df.loc[:, 'shape_oc'].values.squeeze(), specie='OC')

# ozone
for specie in ['CH4', 'N2O', 'Equivalent effective stratospheric chlorine', 'CO', 'VOC', 'NOx']:
    fill(f.species_configs['ozone_radiative_efficiency'], fair_params_df.loc[:, f'o3 {specie}'], specie=specie)

# methane lifetime baseline
fill(f.species_configs['unperturbed_lifetime'], 10.4198121, specie='CH4')

# emissions adjustments for N2O and CH4 
fill(f.species_configs['baseline_emissions'], 19.019783117809567, specie='CH4')
fill(f.species_configs['baseline_emissions'], 0.08602230754, specie='N2O')

# forcing scaling
for specie in ['CH4', 'N2O', 'Stratospheric water vapour', 'Light absorbing particles on snow and ice', 'Land use']:
    fill(f.species_configs['forcing_scale'], fair_params_df.loc[:, f'scale {specie}'].values.squeeze(), specie=specie)
for specie in ['CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18', 'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
    'HFC-365mfc', 'HFC-4310mee']:
    fill(f.species_configs['forcing_scale'], fair_params_df.loc[:, 'scale minorGHG'].values.squeeze(), specie=specie)

# Scale CO2 forcing based on its 4xCO2 calibration
calibrated_f4co2_mean = fair_params_df.loc[:,'F_4xCO2'].values.mean()
fill(
    f.species_configs['forcing_scale'], 
    1 + 0.561*(calibrated_f4co2_mean - fair_params_df.loc[:,'F_4xCO2'].values)/calibrated_f4co2_mean,
    specie='CO2'
)

# tune down volcanic efficacy
fill(f.species_configs['forcing_efficacy'], 0.6, specie='Volcanic')

# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs['baseline_concentration'], 
    fair_params_df.loc[:, 'co2_concentration_1750'].values.squeeze(),
    specie='CO2'
)

# Use interactive methane lifetime
f.ch4_method='Thornhill2021'

In [ ]:
# set initial conditions
initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.airborne_emissions, 0)
initialise(f.cumulative_emissions, 0)

In [ ]:
f.run()

In [ ]:
pl.plot(f.temperature[:,0,:,0] - f.temperature[100:151,0,:,0].mean(axis=0));

In [ ]:
np.median((f.temperature[:,0,:,0] - f.temperature[100:151,0,:,0].mean(axis=0)), axis=1)[245:265].mean()

In [ ]:
f.emissions[250, 0, 0, :]

In [ ]:
first_negative = np.array([f.emissions[:,0,0,2]<0]).argmax()

In [ ]:
f.emissions[first_negative-1:first_negative+1,0,0,2]

In [ ]:
f.cumulative_emissions[first_negative-1:first_negative+1, 0, 0, 2]

In [ ]:
f.cumulative_emissions[-2:,0,0,2].mean() - f.cumulative_emissions[first_negative, 0, 0, 2]